In [1]:
import pandas as pd
import numpy as np
import json
import requests
from warcio import ArchiveIterator

In [2]:
files = pd.read_csv('Data/sept_files.csv', header=None)
files = list(files[0])
len(files)

79600

In [3]:
all_sides = pd.read_csv('Data/allsides.csv')
all_sides.head()

,agree,agree_ratio,agreeance_text,allsides_page,bias,disagree,name,total_votes
0,21052,1.680530,Agrees,https://www.allsides.com/news-source/abc-news-...,left-center,12527,ABC News (Online),33579
1,80,2.857143,Strongly Agrees,https://www.allsides.com/news-source/above-law...,left-center,28,Above The Law,108
2,223,1.581560,Agrees,https://www.allsides.com/news-source/abridge-n...,allsides,141,Abridge News,364
3,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,left,106,AJ+,369
4,4217,0.631382,Disagrees,https://www.allsides.com/news-source/al-jazeer...,left-center,6679,Al Jazeera,10896


In [4]:

def find_url(url):
    return url.replace('https://www.allsides.com/news-source/', '').replace('media-bias', '').replace('-', '')

In [5]:
all_sides['url_match'] = all_sides['allsides_page'].apply(find_url)
all_sides.head()

,agree,agree_ratio,agreeance_text,allsides_page,bias,disagree,name,total_votes,url_match
0,21052,1.680530,Agrees,https://www.allsides.com/news-source/abc-news-...,left-center,12527,ABC News (Online),33579,abcnews
1,80,2.857143,Strongly Agrees,https://www.allsides.com/news-source/above-law...,left-center,28,Above The Law,108,abovelaw
2,223,1.581560,Agrees,https://www.allsides.com/news-source/abridge-n...,allsides,141,Abridge News,364,abridgenews
3,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,left,106,AJ+,369,aj
4,4217,0.631382,Disagrees,https://www.allsides.com/news-source/al-jazeer...,left-center,6679,Al Jazeera,10896,aljazeera


In [6]:
sources = all_sides['url_match'].unique()

In [7]:
text = []
urls = []
t = 0
for file in files[401:500]:
    warc_url = 'https://commoncrawl.s3.amazonaws.com/{}'.format(file)
    wet_url = warc_url.replace('/warc/', '/wet/').replace('warc.gz', 'warc.wet.gz')
    r = requests.get(wet_url, stream=True)
    records = ArchiveIterator(r.raw)
    try:
        i=0
        while i < 100000:
            record = next(records)
            for r1, r2 in record.rec_headers.headers:
                if r1 == 'WARC-Identified-Content-Language' and 'eng' == r2:
                    a = record.content_stream().read()
                    for r1_1, r2_1 in record.rec_headers.headers:
                        if r1_1 == 'WARC-Target-URI':
                            for source in sources:
                                if source in r2_1:
                                    urls.append(source)
                                    text.append(a.decode('utf-8'))
            i += 1
    except:
        t += 1
        print(t)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [8]:
text_df = pd.DataFrame({'text': text,
                       'url_match': urls})
full_data = pd.merge(text_df, all_sides, how='left', on='url_match')
print(full_data.shape)
full_data.head()

(101126, 10)


,text,url_match,agree,agree_ratio,agreeance_text,allsides_page,bias,disagree,name,total_votes
0,Related changes - 2006.igem.org\nChanges relat...,mit,4,1.333333,Somewhat Agrees,https://www.allsides.com/news-source/mit-media...,left,3,Mitú,7
1,Mechanical Behavior of Al-SiCnp Nanocomposite ...,aj,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,left,106,AJ+,369
2,Remote Student Internship experiences at VIC d...,npr,23457,1.249374,Somewhat Agrees,https://www.allsides.com/news-source/npr-media...,center,18775,NPR Online News,42232
3,"Partyware, An Affair To Remember\nHome\nShop\n...",fair,531,1.001887,Somewhat Agrees,https://www.allsides.com/news-source/fair-medi...,left-center,530,FAIR,1061
4,Franchising | Agha Juice\nHome\nFranchising\nL...,aj,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,left,106,AJ+,369


In [9]:
full_data.to_csv('cleaned_data_5.csv', index=False)

In [7]:
dfs = []
for i in list(range(1, 6)):
    print(i)
    df = pd.read_csv('cleaned_data_{}.csv'.format(i))
    print(df.shape)
    dfs.append(df)

1
(102222, 10)
2
(101796, 10)
3
(101553, 10)
4
(102690, 10)
5
(101126, 10)


In [8]:
dfs = pd.concat(dfs)
print(dfs.shape)
dfs.to_csv('full_articles.csv', index=False)

(509387, 10)


In [12]:
df = pd.read_csv(file)
dfs = dfs[dfs['bias'] != 'allsides']

In [13]:
def convert_to_num(target):
    if target == 'left':
        return 0
    elif target == 'left-center':
        return 1
    elif target == 'center':
        return 2
    elif target == 'right':
        return 3
    elif target == 'right-center':
        return 4
    else:
        return np.nan

array(['left', 'left-center', 'center', 'right', 'right-center'],
      dtype=object)

In [ ]:
dfs['target'] = dfs['bais'].apply(convert_to_num)

In [ ]:
train_data, test_data = train_test_split(df, test_size=0.1, random_state=42)
#create count object
count_vectorizer = CountVectorizer(stop_words='english')

#Transform the training data using only the 'text' column values
X_train = count_vectorizer.fit_transform(train_data['text'])

#Transform the test data using only the 'text' column values
X_test = count_vectorizer.transform(test_data['text'])

In [ ]:
tfidf_train_data, tfidf_test_data = train_test_split(df, test_size=0.1, random_state=42)
#create count object
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

#Transform the training data using only the 'text' column values
tfidf_X_train = tfidf_vectorizer.fit_transform(tfidf_train_data['text'])

#Transform the test data using only the 'text' column values
tfidf_X_test = tfidf_vectorizer.transform(tfidf_test_data['text'])

In [ ]:
hash_train_data, hash_test_data = train_test_split(df, test_size=0.1, random_state=42)
#create count object
hash_vectorizer = HashingVectorizer(stop_words='english')

#Transform the training data using only the 'text' column values
hash_X_train = hash_vectorizer.fit_transform(hash_train_data['text'])

#Transform the test data using only the 'text' column values
hash_X_test = hash_vectorizer.transform(hash_test_data['text'])

In [ ]:
y_train = train_data['target']
y_test = test_data['target']
tfidf_y_train = tfidf_train_data['target']
tfidf_y_test = tfidf_test_data['target']
hash_y_train = hash_train_data['target']
hash_y_test = hash_test_data['target']

In [ ]:
def model(train_x, train_y, text_x, test_y, word):
    logit = LogisticRegression(multi_class='multinomial')

    logit = logit.fit(train_x, train_y)
    
    print('Testing Model.')
    y_pred_logit = logit.predict(text_x)

    acc = accuracy_score(test_y, y_pred_logit)
    recall = recall_score(test_y, y_pred_logit, average='weighted')
    perc = precision_score(test_y, y_pred_logit, average='weighted')
    f1 = f1_score(test_y, y_pred_logit, average='weighted')
    
    #model results
    print('Model Results')
    print('Accuracy: {}'.format(acc))
    print('Recall: {}'.format(recall))
    print('Precision: {}'.format(perc))
    print('F1-Score: {}'.format(f1))
    
    return pd.DataFrame({'Word Format': word,
                        'Accuracy': acc,
                        'Recall': recall,
                        'Precision': perc,
                        'F1-Score': f1})
    
    

In [ ]:
results = []

for word in ['Count', 'TFIDF', 'Hash']:
    if word == 'Count':
        results.append(model(X_train, y_train, X_test, y_test, word))
    elif word == 'TFIDF':
        results.append(model(tfidf_X_train, tfidf_y_train, tfidf_X_test, tfidf_y_test, word))
    elif word == 'Hash':
        results.append(model(hash_X_train, hash_y_train, hash_X_test, hash_y_test, word))
    else:
        pass

In [ ]:
results = pd.concat(results)
print(results)